# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import sys
sys.path.append('../')

from unityagents import UnityEnvironment
from collections import deque

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import utils.config 
import pprint
import torch

env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24


### 2. Create instance of agent

Agent's hyperparameters are saved and loaded from config.py file in utils folder. Current values are result of selected after hyperparameter tuing. But you can try different hyperparameter values if you want.

If you just want to see the trained agent jump to cell 5.


In [ ]:
from agents.maddpg import MADDPG

# Load parameters from file
params = utils.config.HYPERPARAMS['Tennis']

# Create agent instance
agent = MADDPG(params['agent'])
print("Created agent with following hyperparameter values:")
pprint.pprint(params['agent'])

Created agent with following hyperparameter values:
{'action_size': 2,
 'agent_name': 'MADDPG',
 'batch_size': 128,
 'buffer_size': 1000000,
 'fc1_units': 128,
 'fc2_units': 150,
 'gamma': 0.999,
 'lr_actor': 0.001,
 'lr_critic': 0.001,
 'n_agents': 2,
 'random_seed': 0,
 'state_size': 24,
 'tau': 1e-05,
 'weight_decay': 0.0001}


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
# Reset and set environment to training mode
env_info = env.reset(train_mode=True)[brain_name]

# Maximum number of training episodes
n_episodes = params['n_episodes']

# List containing scores from each episode
scores = []

# Store last 100 scores
scores_window = deque(maxlen=params['scores_window_size'])

# Filename string
filename = "{:s}_lra{:.0E}_lrc{:.0E}_batch{:d}_fc:{:d}:{:d}_solved{:d}"

# Train loop
for i_episode in range(1, n_episodes+1):
    # Reset environment
    env_info = env.reset(train_mode=True)[brain_name]

    # Observe current state
    states = env_info.vector_observations

    # Reset score
    agent_scores = np.zeros(num_agents)

    # Loop each episode
    while True:

        # Select action with e-greedy policy
        actions = agent.act(states)

        # Take action
        env_info = env.step(actions)[brain_name]

        # Observe the next state
        next_states = env_info.vector_observations

        # Get the reward
        rewards = env_info.rewards

        # Check if episode is finished
        dones = env_info.local_done

        # Store experience
        #agent.step(states, actions, rewards, next_states, dones)

        # State transition
        states = next_states

        # Update total score
        agent_scores += rewards
        
        # Exit loop if episode finished
        if np.any(dones):                                  
            break
            
    # Save most recent score
    scores_window.append(np.max(agent_scores))
    scores.append([np.max(agent_scores), np.mean(scores_window)])

    # Decay epsilon
    #epsilon = max(params['epsilon_final'], params['epsilon_decay']*epsilon)

    # Print learning progress
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
    if i_episode % params['scores_window_size'] == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window)>=params['solve_score']:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        filename = filename.format(hparams['name'], hparams['lr_actor'], hparams['lr_critic'], hparams['batch_size'],
                                   hparams['fc1_units'], hparams['fc2_units'], i_episode-100)
        torch.save(agent.actor_local.state_dict(), 'models/{:s}_actor.pth'.format(filename))
        torch.save(agent.critic_local.state_dict(), 'models/{:s}_critic.pth'.format(filename))
        break

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```